In [6]:
%%configure -f
{ "conf":{    
          "spark.databricks.hive.metastore.glueCatalog.enabled" : "true",
          "spark.jars.packages": "io.delta:delta-core_2.12:0.8.0",
          "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
          "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog",
          "spark.sql.warehouse.dir": "s3://aws-poc-serverless-analytics/dev/clean/delta_dw/"
    
         }
}


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1619432290004_0010,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1619432290004_0010,pyspark,idle,Link,Link,✔


In [7]:
from pyspark.sql.functions import *
from delta.tables import *
spark.sql("SET spark.databricks.delta.schema.autoMerge.enabled = true") 
#import boto3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[key: string, value: string]

In [8]:
target_table = "merchants_cln"
stg_table = "merchants_cln_stg"
database = "delta_lake_poc"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
spark.sql(f"DROP TABLE IF EXISTS {target_table}")
#spark.sql(f"DROP TABLE IF EXISTS {stg_table}")
spark.sql(f"DROP TABLE IF EXISTS merchants_cln_external")
spark.sql(f"DROP DATABASE IF EXISTS {database}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [9]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {database}")
spark.sql(f"USE {database}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [10]:
col_schema_list = spark.sql("""select * from merchants_clnn""").dtypes
col_schema_list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('merchants_id', 'string'), ('company_name', 'string'), ('telephone', 'string'), ('state', 'string'), ('created_at', 'string'), ('updated_at', 'timestamp'), ('active', 'string')]

In [16]:
partition_key_list = ['merchants_id']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
partition_key_schema = ''
for i in partition_key_list:
    for schema in col_schema_list:
        if schema[0] == i:
            #partition_key_schema.append(i + ' ' + schema[1])
            partition_key_schema+=i + ' ' + schema[1] + ','
            col_schema_list.remove(schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
partition_key_schema[:-1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'merchants_id string'

In [19]:
col_schema_list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('company_name', 'string'), ('telephone', 'string'), ('created_at', 'string'), ('updated_at', 'timestamp'), ('active', 'string')]

In [20]:
col_type_schema = ''
for schema in col_schema_list:
    col_type_schema += schema[0] + ' ' + schema[1] + ','

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
col_type_schema[:-1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'company_name string,telephone string,created_at string,updated_at timestamp,active string'

In [17]:
pk_list = ['state']
schema_string = ''
for col,dtype in col_schema_list:
    schema_string+=col + ' ' + dtype + ', '

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
schema_string

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'merchants_id string, company_name string, telephone string, state string, created_at string, updated_at string, active string, mergeKey string, '

In [58]:
col_schema_list = spark.read.csv("s3://aws-poc-serverless-analytics/dev/flattened_raw/mr/",header=True,sep=',',inferSchema=True).dtypes
col_schema_list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('id', 'int'), ('company_name', 'string'), ('telephone', 'string'), ('state', 'string'), ('created_at', 'string'), ('updated_at', 'string')]

In [12]:
inputPath = "s3://aws-poc-serverless-analytics/dev/flattened_raw/mr/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
inputSchema = "id INT, company_name STRING, telephone LONG, state STRING, created_at TIMESTAMP, updated_at TIMESTAMP"

rawDF = (spark.read 
  .option("header", "true")
  .schema(inputSchema)
  .csv(inputPath) 
)
rawDF = rawDF.withColumn('active', lit("true"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
## overwrite
(rawDF.write
  .mode("overwrite")
  .format("delta")
  .partitionBy("state")
  .option("overwriteSchema", "true")
  .saveAsTable(f"{target_table}") )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
## overwrite -- merchants
spark.sql(f"""select * from {target_table}""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+----------+-----------+--------------------+--------------------+------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|active|
+------+--------------------+----------+-----------+--------------------+--------------------+------+
|177863|  Bellelyse Boutique|      null|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|  true|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|  true|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|  true|
|156237|          Washer Fan|      null|provisioned|2020-10-26 20:10:...|2020-10-26 20:53:...|  true|
| 44082|Quick Draw Outdoo...|3182596422|provisioned|2019-02-04 16:57:...|2021-02-10 02:07:...|  true|
|144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|  true|
|140294|              myself|7149259700|       free|2020-09-05 17:41:...|2020-10-0

In [16]:
spark.sql(f"""GENERATE symlink_format_manifest FOR TABLE {target_table}""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [17]:
spark.sql(f""" CREATE EXTERNAL TABLE IF NOT EXISTS {target_table}_external (id INT, company_name STRING, telephone LONG, created_at TIMESTAMP, updated_at TIMESTAMP, active STRING)
PARTITIONED BY (state STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://aws-poc-serverless-analytics/dev/clean/delta_dw/delta_lake_demo.db/merchants_cln/_symlink_format_manifest/' 
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [18]:
spark.sql(f"""MSCK REPAIR TABLE {target_table}_external""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

# Set up relevant paths.

In [30]:
inputPath = "s3a://aws-poc-serverless-analytics/delta_lake_demo/raw/merchants.csv"

workingDir = "s3a://aws-poc-serverless-analytics/delta_lake_demo/clean"

parquetDataPath  = workingDir + "/delta_dw/"
deltaDataPath    = workingDir + "/delta_dw/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
inputSchema = "id INT, company_name STRING, telephone LONG, state STRING, created_at TIMESTAMP, updated_at TIMESTAMP"

rawDF = (spark.read 
  .option("header", "true")
  .schema(inputSchema)
  .csv(inputPath) 
)
rawDF = rawDF.withColumn('active', lit("true"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
rawDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- company_name: string (nullable = true)
 |-- telephone: long (nullable = true)
 |-- state: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)
 |-- active: string (nullable = false)

In [12]:
target_table = "merchants_cln"
stg_table = "merchants_cln_stg"
database = "delta_lake_demo"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
spark.sql(f"DROP TABLE IF EXISTS {target_table}")
spark.sql(f"DROP TABLE IF EXISTS {stg_table}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [29]:
spark.sql(f"DROP DATABASE IF EXISTS {database}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.hadoop.hive.ql.metadata.HiveException: MetaException(message:Unable to drop database: InvalidOperationException(message:Database delta_lake_demo is not empty.));
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 649, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1305, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 134, in deco
    raise_from(converted)
  File "<string>", line 3, in raise_from
pyspark.sql.utils.AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: MetaException(message:Unable to drop database: InvalidOperationException(message:Database delta_lake_demo is not empty.));



In [13]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {database}")
spark.sql(f"USE {database}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]


###  WRITE to Delta


In [42]:
# write using Databricks Delta format
(rawDF.write
  .mode("overwrite")
  .format("delta")
  .partitionBy("state")
  .option("overwriteSchema", "true")
  .saveAsTable(f"{target_table}") )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
spark.sql(f"""select * from {target_table}""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+----------+-----------+--------------------+--------------------+------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|active|
+------+--------------------+----------+-----------+--------------------+--------------------+------+
|177863|  Bellelyse Boutique|      null|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|  true|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|  true|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|  true|
|144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|  true|
|140294|              myself|7149259700|       free|2020-09-05 17:41:...|2020-10-06 11:15:...|  true|
|156444|     Only The Flyest|4049038447|       free|2020-10-27 05:26:...|2020-11-26 11:15:...|  true|
|156237|          Washer Fan|      null|provisioned|2020-10-26 20:10:...|2020-10-2

In [146]:
spark.sql(f"""GENERATE symlink_format_manifest FOR TABLE {target_table}""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [120]:
spark.sql(f""" CREATE EXTERNAL TABLE {target_table}_external (id INT, company_name STRING, telephone LONG, created_at TIMESTAMP, updated_at TIMESTAMP, active STRING)
PARTITIONED BY (state STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://aws-poc-serverless-analytics/delta_lake_demo/clean/delta_dw/delta_lake_demo.db/merchants_cln/_symlink_format_manifest/' 
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Table or view 'merchants_cln_external' already exists in database 'delta_lake_demo';
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 649, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1305, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 134, in deco
    raise_from(converted)
  File "<string>", line 3, in raise_from
pyspark.sql.utils.AnalysisException: Table or view 'merchants_cln_external' already exists in database 'delta_lake_demo';



In [147]:
spark.sql(f"""MSCK REPAIR TABLE {target_table}_external""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

## Read in Some New Data for SCD 1

In [8]:
miniDataInputPath = "s3a://aws-poc-serverless-analytics/delta_lake_demo/raw/merchants_SCD1.txt"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
inputSchema = "id INT, company_name STRING, telephone LONG, state STRING, created_at TIMESTAMP, updated_at TIMESTAMP"

newDataDF = (spark.read 
  .option("header", "true")
  .schema(inputSchema)
  .csv(miniDataInputPath) 
)
newDataDF = newDataDF.withColumn('active', lit("true"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# write using Databricks Delta format
(newDataDF.write
  .mode("overwrite")
  .format("delta")
  .option("overwriteSchema","true")
  .partitionBy("state")
  .saveAsTable(f"{stg_table}") )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
spark.sql(f"""select * from {stg_table}""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+----------+-----------+--------------------+--------------------+------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|active|
+------+--------------------+----------+-----------+--------------------+--------------------+------+
|177863|  Bellelyse Boutique|      null|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|  true|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|  true|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|  true|
|144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|  true|
|140294|              myself|7149259700|       free|2020-09-05 17:41:...|2020-10-06 11:15:...|  true|
|156444|     Only The Flyest|4049038447|       free|2020-10-27 05:26:...|2020-11-26 11:15:...|  true|
|156237|          Washer Fan|      null|provisioned|2020-10-26 20:10:...|2020-10-2

In [17]:
spark.sql(f""" 
              MERGE INTO {target_table}
              USING {stg_table}
              on {target_table}.id = {stg_table}.id
              WHEN MATCHED THEN 
                  UPDATE SET *
              WHEN NOT MATCHED 
                  THEN INSERT *
              
            """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [18]:
spark.sql(f"""select * from {target_table}""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+----------+-----------+--------------------+--------------------+------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|active|
+------+--------------------+----------+-----------+--------------------+--------------------+------+
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|  true|
| 44082|Quick Draw Outdoo...|3182596422|provisioned|2019-02-04 16:57:...|2021-02-10 02:07:...|  true|
|156444|     Only The Flyest|4049038447|       free|2020-10-27 05:26:...|2020-11-26 11:15:...|  true|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|  true|
|165163|  JeeJee’s Boutique |      null|trial_ended|2020-11-20 16:31:...|2020-11-20 16:33:...|  true|
|144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|  true|
|178307|            MSI Kart|9319919988|trial_ended|2021-01-02 09:33:...|2021-01-0

In [128]:
spark.sql(f"""GENERATE symlink_format_manifest FOR TABLE {target_table}""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [ ]:
spark.sql(f"""MSCK REPAIR TABLE {target_table}_external""")

## Read in Some New Data for SCD 2 

In [148]:
SCD2_Path = "s3a://aws-poc-serverless-analytics/delta_lake_demo/raw/merchants_SCD2.txt"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [149]:
inputSchema = "id INT, company_name STRING, telephone LONG, state STRING, created_at TIMESTAMP, updated_at TIMESTAMP"

rawDF = (spark.read 
  .option("header", "true")
  .schema(inputSchema)
  .csv(SCD2_Path) 
)
rawDF = rawDF.withColumn('active', lit("true"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [150]:
# write using Databricks Delta format
(rawDF.write
  .mode("overwrite") 
  .format("delta")
  .partitionBy("state")
  .option("overwriteSchema","true")
  .saveAsTable(f"{stg_table}") )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [151]:
spark.sql(f""" MERGE INTO {target_table}
USING (
   SELECT {stg_table}.id as mergeKey, {stg_table}.*
  FROM {stg_table}
  
  UNION ALL
    SELECT NULL as mergeKey, {stg_table}.*
  FROM {stg_table} JOIN {target_table}
  ON {stg_table}.id = {target_table}.id 
  WHERE {target_table}.active = true AND {stg_table}.updated_at <> {target_table}.updated_at 
  
) staged_updates
ON {target_table}.id = mergeKey
WHEN MATCHED AND {target_table}.active = true AND {target_table}.updated_at <> staged_updates.updated_at THEN  
  UPDATE SET active = false
WHEN NOT MATCHED THEN 
  INSERT *
            """)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [152]:
spark.sql(f"""select * from {stg_table}""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+----------+-----------+--------------------+--------------------+------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|active|
+------+--------------------+----------+-----------+--------------------+--------------------+------+
|177863|  Bellelyse Boutique|      null|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|  true|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|  true|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|  true|
| 44082|Quick Draw Outdoo...|3182596422|  suspended|2019-02-04 16:57:...|2021-02-11 02:07:...|  true|
|144524|     Rachid hamzaoui| 639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|  true|
|140294|              myself|7149259700|       free|2020-09-05 17:41:...|2020-10-06 11:15:...|  true|
|156444|     Only The Flyest|4049038447|       free|2020-10-27 05:26:...|2020-11-2

In [153]:
spark.sql(f"""select * from {target_table}""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+----------+-----------+--------------------+--------------------+------+--------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|active|mergeKey|
+------+--------------------+----------+-----------+--------------------+--------------------+------+--------+
| 44082|Quick Draw Outdoo...|3182596422|  suspended|2019-02-04 16:57:...|2021-02-11 02:07:...|  true|    null|
| 44082|Quick Draw Outdoo...|3182596422|provisioned|2019-02-04 16:57:...|2021-02-10 02:07:...| false|    null|
|156237|          Washer Fan|      null|provisioned|2020-10-26 20:10:...|2020-10-26 20:53:...|  true|    null|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|  true|    null|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|  true|    null|
|165163|  JeeJee’s Boutique |      null|trial_ended|2020-11-20 16:31:...|2020-11-20 16:33:...|  true|    null|
|

In [155]:
spark.sql(f"""GENERATE symlink_format_manifest FOR TABLE {target_table}""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [156]:
spark.sql(f"""MSCK REPAIR TABLE {target_table}_external""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]